In [231]:
import pandas as pd
import numpy as np

# Carregando os dados
caminho = "C:\\Users\\natal\\OneDrive\\Área de Trabalho\\Estudo Python\\SistemaRecomendacao\\ml-latest-small"
caminho_arquivo = caminho + "\\movies.csv"
filmes = pd.read_csv(caminho_arquivo)
filmes.columns = ["filmeId", "titulo", "genero"]
filmes = filmes.set_index("filmeId")

caminho_arquivo = caminho + "\\ratings.csv"  
notas = pd.read_csv(caminho_arquivo)
notas.columns = ["usuarioId", "filmeId", "nota", "momento"]

In [232]:
notas.head()

,usuarioId,filmeId,nota,momento
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [233]:
len(filmes)

9742

In [234]:
# Calculando o total de votos por filme e a média das notas por filme
total_de_votos = notas["filmeId"].value_counts()
filmes["total_votos"] = total_de_votos

media_das_notas = notas.groupby("filmeId").mean()["nota"]
filmes["media_das_notas"] = media_das_notas


filmes_com_mais_de_cinquenta_votos = filmes.query("total_votos> 50")
notas = notas.set_index("filmeId").loc[filmes_com_mais_de_cinquenta_votos.index]
notas = notas.reset_index()

In [235]:
total_de_votos.head()

filmeId
356     329
318     317
296     307
593     279
2571    278
Name: count, dtype: int64

In [236]:
filmes.query("total_votos >= 10").sort_values("media_das_notas", ascending = False).head(10)

,titulo,genero,total_votos,media_das_notas
filmeId,,,,
1041,Secrets & Lies (1996),Drama,11.0,4.590909
3451,Guess Who's Coming to Dinner (1967),Drama,11.0,4.545455
1178,Paths of Glory (1957),Drama|War,12.0,4.541667
1104,"Streetcar Named Desire, A (1951)",Drama,20.0,4.475000
2360,"Celebration, The (Festen) (1998)",Drama,12.0,4.458333
1217,Ran (1985),Drama|War,15.0,4.433333
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
951,His Girl Friday (1940),Comedy|Romance,14.0,4.392857
1927,All Quiet on the Western Front (1930),Action|Drama|War,10.0,4.350000


In [237]:
def notas_do_usuario(usuario):
    notas_do_usuario = notas.query("usuarioId==%d" % usuario)
    notas_do_usuario = notas_do_usuario[["filmeId", "nota"]].set_index(["filmeId"])
    return notas_do_usuario

In [238]:
filmes.head()

,titulo,genero,total_votos,media_das_notas
filmeId,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0,3.920930
2,Jumanji (1995),Adventure|Children|Fantasy,110.0,3.431818
3,Grumpier Old Men (1995),Comedy|Romance,52.0,3.259615
4,Waiting to Exhale (1995),Comedy|Drama|Romance,7.0,2.357143
5,Father of the Bride Part II (1995),Comedy,49.0,3.071429


In [239]:
notas_do_usuario(1)

,nota
filmeId,
1,4.0
3,4.0
6,4.0
47,5.0
50,5.0
...,...
3489,4.0
3527,4.0
3578,5.0


In [240]:
def distancia_de_vetores(a,b):
    return np.linalg.norm(a - b)

In [241]:
def distancia_de_usuarios(usuarioId1, usuarioId2, minimo = 5):
    notas1 = notas_do_usuario(usuarioId1)
    notas2 = notas_do_usuario(usuarioId2)
    diferencas = notas1.join(notas2, lsuffix = "_esquerda", rsuffix = "_direita").dropna()
    if(len(diferencas) < minimo):
       return None
    distancia = distancia_de_vetores(diferencas['nota_esquerda'], diferencas['nota_direita'])
    return [usuarioId1, usuarioId2, distancia]

In [242]:
distancia_de_usuarios(1,2)

In [243]:
def distancia_de_todos(voce_id, numero_de_usuarios_a_analisar  = None):
    todos_os_usuarios = notas['usuarioId'].unique()
    if numero_de_usuarios_a_analisar:
        todos_os_usuarios = todos_os_usuarios[:numero_de_usuarios_a_analisar ]
    distancias = [distancia_de_usuarios(voce_id, usuario_Id) for usuario_Id in todos_os_usuarios]
    distancias = list(filter(None, distancias))
    distancias = pd.DataFrame(distancias, columns = ["você", "outra_pessoa", "distancia"])
    return distancias

In [244]:
distancia_de_todos(1, 50)

,você,outra_pessoa,distancia
0,1,1,0.000000
1,1,5,3.741657
2,1,7,7.106335
3,1,15,5.291503
4,1,17,4.330127
5,1,18,8.000000
6,1,19,12.328828
7,1,21,8.789198
8,1,27,7.071068
9,1,31,4.582576


In [245]:
def knn(voce_id, k_mais_proximos=10, numero_de_usuarios_a_analisar = None):
  distancias = distancia_de_todos(voce_id, numero_de_usuarios_a_analisar = numero_de_usuarios_a_analisar)
  distancias = distancias.sort_values("distancia")
  distancias = distancias.set_index("outra_pessoa").drop(voce_id, errors='ignore')
  return distancias.head(k_mais_proximos)

In [246]:
def sugere_para(voce, k_mais_proximos = 10, numero_de_usuarios_a_analisar = None):
  notas_de_voce = notas_do_usuario(voce)
  filmes_que_voce_ja_viu = notas_de_voce.index

  similares = knn(voce, k_mais_proximos = k_mais_proximos, numero_de_usuarios_a_analisar = numero_de_usuarios_a_analisar)
  usuarios_similares = similares.index
  notas_dos_similares = notas.set_index("usuarioId").loc[usuarios_similares]
  recomendacoes = notas_dos_similares.groupby("filmeId").mean()[["nota"]].rename(columns={"nota": "nota_media"})
  aparicoes = notas_dos_similares.groupby("filmeId").count()[['nota']].rename(columns={"nota": "aparicoes"})
  
  filtro_minimo = k_mais_proximos / 2
  recomendacoes = recomendacoes.join(aparicoes)
  recomendacoes = recomendacoes.query("aparicoes >= %.2f" % filtro_minimo)  
  recomendacoes = recomendacoes.sort_values("nota_media", ascending=False)
  recomendacoes = recomendacoes.drop(filmes_que_voce_ja_viu,errors='ignore')
  return recomendacoes.join(filmes)
  """return recomendacoes"""

In [247]:
sugere_para(1)

,nota_media,aparicoes,titulo,genero,total_votos,media_das_notas
filmeId,,,,,,
7153,4.750000,6,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy,185.0,4.118919
58559,4.750000,6,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX,149.0,4.238255
4993,4.714286,7,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy,198.0,4.106061
318,4.583333,6,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
109487,4.583333,6,Interstellar (2014),Sci-Fi|IMAX,73.0,3.993151
79132,4.357143,7,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX,143.0,4.066434


In [248]:
sugere_para(1, numero_de_usuarios_a_analisar=500)

,nota_media,aparicoes,titulo,genero,total_votos,media_das_notas
filmeId,,,,,,
318,4.583333,6,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
